In [1]:
import sys
import subprocess
import importlib
from abc import ABC, abstractmethod
import random 


# ------------------------------------ Checking for external libs if exists or install and import them -------------------------------------- #

# Creating a function for installing and import the tabulate 
def install_and_import_from(module, function_or_class):
    try:
        # Try to dynamically import the module
        mod = importlib.import_module(module)
        # Try to import the specific function/class
        globals()[function_or_class] = getattr(mod, function_or_class)
    except ImportError:
        # Install the module if not found
        print(f"{module} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", module])
        # Retry the import after installation
        mod = importlib.import_module(module)
        globals()[function_or_class] = getattr(mod, function_or_class)
    except AttributeError:
        print(f"{function_or_class} not found in the {module} module.")


# ------------------------------------ Creating random items for our shop -------------------------------------- #


random.seed(10)   # Setting random.seed in order to generate every time the same 'random' results

# Creating a list of category items
clothing = ['T-shirts', 'Jeans', 'Hoodies and Sweatshirts', 'Underwear']

# Importing random details about product_id, product_name, price, size
def generate_details(category, products):
    product_list = []  # Initialized an empty list
    for i in range(1, products + 1):
        product_id = f'{category[:1]}{random.randint(0000, 9999)}'   # Creating id's in format eg. "J0208"
        product_name = f'{category} {i}'   # Creating names in format eg. "Jeans 1"
        price = round(random.uniform(75, 750))    
        size = random.choice(['S', 'M', 'L', 'XL', 'XXL'])
        stock = round(random.uniform(0, 50))
        product_list.append({'Product ID':product_id, 'Product Name':product_name, 'Price':price, 'Size':size, 'Stock':stock})  
    return product_list

# Generate product details for each category of list clothing
product_catalog = []

for category in clothing:
    products = random.randint(1, 50)  # Random number of products per category
    product_catalog.extend(generate_details(category, products))  # Assign each product on the list product_catalog


# Initialize the list in which all products will be added here
# Placing it outside of all classes and functions make it global and readable to everything
order_placement = []  

    
class Menu(ABC):
    def __init__(self):
        pass

    @abstractmethod  
    def show_menu(self):
        pass


# Creating class Display in order user to be able to display the whole catalog or search for specific item from the main menu
class Display(Menu):
    
    def __init__(self):
        pass

    def show_menu(self):
        while True:
            print('1. To display all products')
            print('2. To search a product')
            print('3. To go back to the main menu\n')
            choice = input('Choose an option:\n')

            if choice == '1':
                self.display_all_products()
            elif choice == '2':
                self.search_product()
            elif choice == '3':
                break
            else:
                print("Invalid option. Please try again.\n")

    def display_all_products(self):
        table = tabulate(product_catalog, headers='keys', tablefmt='grid')
        print(table)

    # Implementing linear search to find products in the catalog. 
    # This allows the user to search by Product ID or Product Name
    def search_product(self):
        
        print('\nSearch by:')
        print('Press:\n')
        print('1. Product ID')
        print('2. Product Name\n')
        choice = input('Choose an option:\n')

        if choice == '1':
            search_key = 'Product ID'
        elif choice == '2':
            search_key = 'Product Name'
        else:
            print('Invalid option. Returning to menu.\n')
            return

        search_value = input(f'Enter the {search_key}: ').strip()
        search_results = []

        # Linear search through the product catalog
        for product in product_catalog:
            if product.get(search_key, '').lower() == search_value.lower():
                search_results.append(product)

        if search_results:
            print("\nSearch Results:")
            print(tabulate(search_results, headers='keys', tablefmt='grid'))
        else:
            print(f"\nNo products found with {search_key} matching '{search_value}'.\n")


# ------------------------------------ Handling Management level operations from Admin -------------------------------------- #


class Product(Menu):
    def __init__(self):
        pass

    def show_menu(self):
        choice = input('Please type the password: ')
        while choice == 'GiorgosTzimas':
            print('\nPress:\n')
            print('1. To add a new product')
            print('2. To remove a product from the catalog')
            print('3. To update the price of a current product')
            print('4. To update the size of a current product')
            print('5. To update the stock of a current product')
            print('6. To go back to main menu\n')
            choice2 = input('Choose one of the following options\n')

            if choice2 == '1':
                self.add_product()
            elif choice2 == '2':
                self.remove_product()
            elif choice2 == '3':
                self.update_price()
            elif choice2 == '4':
                self.update_size()
            elif choice2 == '5':
                self.update_stock()
            elif choice2 == '6':
                break
            else:
                print('Not available operation\n')
        
    
    # If new product needs to be added:
    def add_product(self):
        print('Please type: Product ID, Product Name, Price, Size, Stock\n')
        
        while True:
            # Ask for user input
            choice1 = input('Enter the details separated by commas or type "0" to return: ').strip()

            # Check if user wants to return to the previous menu
            if choice1.lower() == '0':
                print('Returning to the previous menu...\n')
                break  # Exit the loop

            try:
                # Split input into fields and strip whitespace
                product_id, product_name, price, size, stock = [item.strip() for item in choice1.split(',')]

                # Convert price and stock to appropriate types
                price = int(price)  # Ensure price is a number
                stock = int(stock)    # Ensure stock is an integer

                # Check if product ID already exists
                for product in product_catalog:
                    if product['Product ID'] == product_id:
                        print(f'Product with ID {product_id} already exists!')
                        return  # Exit the method

                 # Add new product to the catalog
                product_catalog.append({
                    'Product ID': product_id,
                    'Product Name': product_name,
                    'Price': price,
                    'Size': size,
                    'Stock': stock
                })
                print(f'Product {product_id} added successfully!\n')

            except ValueError:
                print('Invalid input. Please ensure all details are provided and formatted correctly (e.g., P004, T-Shirt, 200, M, 155)')

    # If a product need to be entirely removed from the product_catalog
    def remove_product(self):
        while True:
            choice = input('Please type the code of the product you want to remove or press "0" to return\n')
           
            if choice.lower() == '0':
                print('Returning to the previous menu...\n')
                break  # Exit the loop
            else:
                item_found = False
                error_message = None
                for product in product_catalog:
                    try:
                        if choice == product['Product ID']:
                            product_catalog.remove(product)  # Remove the product from the catalog
                            print(f'Product with ID {product['Product ID']} has been removed.\n') 
                            item_found = True
                            break  # If it succesfully finds the product exit the loop
                        elif not item_found:
                            raise ValueError(f'Product with ID {choice} does not exist.\n')
                    except ValueError as e:
                        error_message = e
                if error_message is not None and item_found == False:
                    print(error_message)

    # If only the price has to be updated
    def update_price(self):
        while True:
            choice = input('Please type the code of the product you want to modify or press "0" to return\n')
        
            if choice.lower() == '0':
                print('Returning to the previous menu...\n')
                break  # Exit the loop
            else:
                # Check if the product exists in the catalog
                item_found = False
                for product in product_catalog:
                    if choice == product['Product ID']:
                        item_found = True
                        try:
                            new_price = int(input('Please type the new price you want to replace: '))
                            product['Price'] = new_price
                            print(f'The price of the product {choice} has been updated successfully to {product['Price']}\n')
                            break
                        except ValueError:
                            print('Invalid input. Please enter a valid integer price.\n')
                            
                if not item_found:
                    print(f'Product with ID {choice} does not exist.\n')
                   
    # If only the size has to be updated   
    def update_size(self):  # Same with sizes + incase that something hasn't the correct size and needs to be changed
        size_list = ['S', 'M', 'L', 'XL', 'XXL']
        while True:
            choice = input('Please type the code of the product you want to modify or press "0" to return\n')
            
            if choice.lower() == '0':
                print('Returning to the previous menu...\n')
                break  # Exit the loop
            else:
                # Check if the product exists in the catalog
                item_found = False
                for product in product_catalog:
                    if choice == product['Product ID']:
                        item_found = True
                        new_size = input('Please type the new size you want to replace: ')
                        if new_size in size_list:
                            product['Size'] = new_size
                            print(f'The size of the product {choice} has been updated successfully to {product['Size']}\n')
                            break
                        else:
                            print(f'Invalid input. Please enter a valid size from the list {size_list}.\n')
                            
                if not item_found:
                    print(f'Product with ID {choice} does not exist.\n')
                    

    # If only the stock has to be updated
    def update_stock(self):
        while True:
            choice = input('Please type the code of the product you want to modify or press "0" to return\n')
        
            if choice.lower() == '0':
                print('Returning to the previous menu...\n')
                break  # Exit the loop
            else:
                # Check if the product exists in the catalog
                item_found = False
                for product in product_catalog:
                    if choice == product['Product ID']:
                        item_found = True
                        try:
                            new_stock = int(input('Please type the new stock you want to replace: '))
                            product['Stock'] = new_stock
                            print(f'The price of the product {choice} has been updated successfully to {product['Stock']}\n')
                            break
                        except ValueError:
                            print('Invalid input. Please enter a valid integer number for stock.\n')
                            
                if not item_found:
                    print(f'Product with ID {choice} does not exist.\n')            


# ------------------------------------------ Handling Order level operations ------------------------------------------ #

    
class Order(Menu):    
    def __init__(self, order_id, customer_id):
        self.order_id = order_id
        self.customer_id = customer_id
        

    def show_menu(self):
        while True:
            print('Press:')
            print('1. To add a Product Code')
            print('2. To remove a Product Code you want')
            print('3. To go back to main menu\n')
            choice = input('Choose an option')

            if choice == '1':
                print('\nLoading...\n')
                table = tabulate(product_catalog, headers = 'keys', tablefmt = 'grid')
                print(table)
                self.add_item()
            elif choice == '2':
                self.show_added_products()
                self.remove_item(input('Delete one of the following you have already added'))
            elif choice == '3':
                print('Back to main menu')
                break

    
    def add_item(self):
        while True:
            product_id_add = input("Please type the product you desire or press 'f' to finish: ")
            
            if product_id_add.lower() == 'f':   # Check for 'f' to exit the loop
                print("Finished adding products.")
                break

            if not product_id_add:
                print('Invalid input. Please add a valid product id')
                continue
                
            product_found = False   # Validate product ID in the catalog
            for product in product_catalog:
                if product['Product ID'] == product_id_add:
                    if product['Stock'] > 0:    # If the item is in stock, it adds it on the order_placement list
                        order_placement.append(product)
                        product['Stock'] -= 1    # Remove 1 item from the stock
                        print(f'The product {product_id_add} has successfully been added.')
                        product_found = True
                    else:
                        print(f'The item {product_id_add} is out of stock')
        
            if not product_found:
                print(f'The product {product_id_add} either does not exist or is out of stock.')

    def remove_item(self, product_id):
        for product in order_placement:
            if product['Product ID'] == product_id:
                order_placement.remove(product)  # Remove the product from order placement
                for catalog_product in product_catalog:
                    if catalog_product['Product ID'] == product_id:
                        catalog_product['Stock'] += 1  # Update stock in catalog
                        print(f'Product {product_id} removed from the order and stock has been updated.')
                        return
    

    def show_added_products(self):
        table_order = tabulate(order_placement, headers = 'keys', tablefmt = 'grid')
        print(table_order)
        

    
class Shopping_Cart(Order, Menu):
    
    def __init__(self, order_id, customer_id):
        super().__init__(order_id, customer_id)


    def show_menu(self):
        while True:
            print('Press:')
            print('1. My Shopping Cart')
            print('2. To apply discount')
            print('3. To Checkout')
            print('4. To go back to main menu\n')
            choice = input('Choose one of the following options')
            
            if choice == '1':
                print(self.show_added_products())
            elif choice == '2':
                self.apply_discount()
            elif choice == '3':
                self.checkout()
            elif choice == '4':
                break

    
    def apply_discount(self):
        if not order_placement:
            print("Your cart is empty. Add items before applying a discount.")
            return

        print('Do you wanna apply a discount code?')
        answer = input('yes/no')
        if answer == 'yes':
            discount_code = input('Enter the discount code: ')
            if discount_code == 'WINTER15':  # The only running discount code right now
                total = 0
                for item in order_placement:
                    total += item['Price']
                    discounted_total = total * 0.85  # 15% discount
                print(f'Discount applied! Your total after discount is: {round(discounted_total,2)} DKK.\n')
            else:
                print('Invalid discount code.')
        else:
            print(f'Your total after discount is: {round(total,2)} DKK.\n')
        

    def display_order(self):
        if not order_placement:  # Check if cart is empty
            print('Your shopping cart is empty.')
        else:
            print('Items in your shopping cart:')
            print(tabulate(order_placement, headers = 'keys', tablefmt = 'grid'))

    
    def checkout(self):
        if not order_placement:
            print('Your cart is empty. Add Items before checkout.\n')
            return
        else:
            print('\nProceeding to checkout...\n')
            print('Please enter your personal details:')
            name = input('Enter your name: ').strip()
            email = input('Enter your email: ').strip()
            phone = input('Enter your phone number: ').strip()
    
            # Example receipt
            print('\n--- Order Confirmation ---')
            print(f'Customer Name: {name}')
            print(f'Email: {email}')
            print(f'Phone: {phone}')
  
            # Clear the order placement list after successful checkout
            order_placement.clear()
            print("\nOrder placed successfully! Thank you for your purchase. We have sent you an email of your purchase information. We hope to see you again!\n")
    
    
# --------------------------- Functions outside of the classes to make the program run effectively --------------------------- #

    
def show_main_menu():
        print('Welcome to the store!\n')
        print('Fixed 15% discount to our all products with the code: WINTER15\n')
        print('Press:')
        print('1. For Purchase')
        print('2. For Shopping Cart')
        print('3. To see our products')
        print('4. To enter as Admin')
        print('5. To Exit\n')


def run():
    menus = {'1': Order(1,1), '2': Shopping_Cart(1,1), '3': Display(), '4': Product()}
    
    while True:
        show_main_menu()
        choice = input('Choose an option:')
            
        if choice in menus:
            menu = menus[choice]
            menu.show_menu()
        elif choice == '5':
            print('Exiting from the store...')
            break
        else:
            print('Not available operation')


# --------------------------------------------------- Run the program ------------------------------------------------------ #

# Install and Import tabulate
install_and_import_from("tabulate", "tabulate")

begin=run()

begin

Welcome to the store!

Fixed 15% discount to our all products with the code: WINTER15

Press:
1. For Purchase
2. For Shopping Cart
3. To see our products
4. To enter as Admin
5. To Exit



Choose an option: 1


Press:
1. To add a Product Code
2. To remove a Product Code you want
3. To go back to main menu



Choose an option 1



Loading...

+--------------+----------------------------+---------+--------+---------+
| Product ID   | Product Name               |   Price | Size   |   Stock |
+==============+============================+=========+========+=========+
| T533         | T-shirts 1                 |     365 | XXL    |       1 |
+--------------+----------------------------+---------+--------+---------+
| T7578        | T-shirts 2                 |     624 | L      |      33 |
+--------------+----------------------------+---------+--------+---------+
| T2625        | T-shirts 3                 |      98 | XL     |      16 |
+--------------+----------------------------+---------+--------+---------+
| T4095        | T-shirts 4                 |     718 | L      |       2 |
+--------------+----------------------------+---------+--------+---------+
| T2272        | T-shirts 5                 |     482 | XL     |      21 |
+--------------+----------------------------+---------+--------+---------+
| T4297     

...

Press:
1. To add a Product Code
2. To remove a Product Code you want
3. To go back to main menu



Choose an option 3


Back to main menu
Welcome to the store!

Fixed 15% discount to our all products with the code: WINTER15

Press:
1. For Purchase
2. For Shopping Cart
3. To see our products
4. To enter as Admin
5. To Exit



Choose an option: 5


Exiting from the store...
